<a href="https://colab.research.google.com/github/leejh12345/ai/blob/main/crab_age_prediction/2024_AI_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kaggle에서 baseline 코드가 안보이는 관계로 부득이하게 colab에서 실행   

files.upload()까지 코드가 진행되면, kaggle login창이 나타나고 로그인 진행   
이전에 kaggle 사이트에서 프로필 사진 클릭 후 -> settings 진입 -> create new token
여기서 생성된 kaggle.json을 업로드
그러면 private 데이터라고 하더라도 현재 colab으로 들고올 수 있음


In [ ]:
!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c 2024-csu-iceai-crab-age-regression-competition

!mkdir -p /kaggle/input/2024-csu-iceai-crab-age-regression-competition
!mkdir -p /kaggle/output/2024-csu-iceai-crab-age-regression-competition
!unzip 2024-csu-iceai-crab-age-regression-competition.zip -d /kaggle/input/2024-csu-iceai-crab-age-regression-competition

2024-csu-iceai-crab-age-regression-competition.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2024-csu-iceai-crab-age-regression-competition.zip
  inflating: /kaggle/input/2024-csu-iceai-crab-age-regression-competition/sample_submission.csv  
  inflating: /kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv  
  inflating: /kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv  


## 환경 설정   
라이브러리 설치   
pandas, scikt-learn

In [ ]:
!pip install pandas scikit-learn

## 0. 필요한 라이브러리 호출

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
!ls /kaggle/input/2024-csu-iceai-crab-age-regression-competition

sample_submission.csv  test_v2.csv  train.csv


## 1-1. 데이터셋 로드

In [ ]:
data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv')

print(data.head())

   id Sex  Length  Diameter  Height     Weight  Shucked Weight  \
0   0   M  1.6375     1.300  0.4750  38.711242       17.690088   
1   1   M  1.5375     1.175  0.3750  33.409886       11.495722   
2   2   F  1.7125     1.375  0.4750  43.629880       16.726205   
3   3   F  1.6375     1.275  0.4375  38.980562       16.683681   
4   4   F  1.7125     1.375  0.4250  51.511041       22.254358   

   Viscera Weight  Shell Weight   Age  
0        7.413394     11.339800  10.0  
1        8.022908      8.930093  17.0  
2       11.424848     14.033003  13.0  
3        6.676307     11.580771  11.0  
4       11.439023     12.332033  10.0  


## 1-2. 데이터셋 전처리

In [ ]:
data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv')

# 데이터 전처리 추가
from sklearn.preprocessing import LabelEncoder

# Sex 컬럼 인코딩
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
test_data['Sex'] = le.fit_transform(test_data['Sex'])

# 추가: 이상치 제거 (예: Weight 열)
Q1 = data['Weight'].quantile(0.25)
Q3 = data['Weight'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data = data[(data['Weight'] >= lower_bound) & (data['Weight'] <= upper_bound)]

# 추가: 새로운 변수 생성 (예: Volume, Density)
data['Volume'] = data['Length'] * data['Diameter'] * data['Height']
data['Density'] = data['Weight'] / data['Volume']
test_data['Volume'] = test_data['Length'] * test_data['Diameter'] * test_data['Height']
test_data['Density'] = test_data['Weight'] / test_data['Volume']

# 추가 변수 생성
data['Weight_to_Length'] = data['Weight'] / data['Length']
data['Shucked_to_Weight'] = data['Shucked Weight'] / data['Weight']
data['Viscera_to_Weight'] = data['Viscera Weight'] / data['Weight']
data['Length_to_Height'] = data['Length'] / (data['Height'] + 1e-8)  # Divide by zero 방지

test_data['Weight_to_Length'] = test_data['Weight'] / test_data['Length']
test_data['Shucked_to_Weight'] = test_data['Shucked Weight'] / test_data['Weight']
test_data['Viscera_to_Weight'] = test_data['Viscera Weight'] / test_data['Weight']
test_data['Length_to_Height'] = test_data['Length'] / (test_data['Height'] + 1e-8)

In [ ]:
# 1-2. 데이터셋 전처리
X = data[['Sex', 'Length', 'Diameter', 'Height', 'Weight',
          'Shucked Weight', 'Viscera Weight', 'Shell Weight',
          'Volume', 'Density', 'Weight_to_Length',
          'Shucked_to_Weight', 'Viscera_to_Weight', 'Length_to_Height']]
y = data['Age']

X_test = test_data[['Sex', 'Length', 'Diameter', 'Height', 'Weight',
                    'Shucked Weight', 'Viscera Weight', 'Shell Weight',
                    'Volume', 'Density', 'Weight_to_Length',
                    'Shucked_to_Weight', 'Viscera_to_Weight', 'Length_to_Height']]

# 데이터셋 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 무한대 값과 NaN 값 처리하기
import numpy as np

# 무한대 값을 NaN으로 변환
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_valid.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균으로 대체 (또는 다른 방법을 사용할 수 있음)
X_train.fillna(X_train.mean(), inplace=True)
X_valid.fillna(X_valid.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

# 스케일링 (예: MinMaxScaler로 변경)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

## 3.a. Decision tree를 이용한 regression model

In [ ]:
dt_regressor = DecisionTreeRegressor(random_state = 42)
dt_regressor.fit(X_train, y_train)
y_pred_dt = dt_regressor.predict(X_valid)

## 3.b. support vector machine를 이용한 regression model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

svm_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf']
}

svm_search = RandomizedSearchCV(
    SVR(),
    param_distributions=svm_params,
    n_iter=10, scoring='neg_mean_absolute_error', cv=3, random_state=42
)
svm_search.fit(X_train_scaled, y_train)
svm_regressor = svm_search.best_estimator_

y_pred_svm = svm_regressor.predict(X_valid_scaled)

In [ ]:
# 추가: Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=rf_params,
    n_iter=10, scoring='neg_mean_absolute_error', cv=3, random_state=42
)
rf_search.fit(X_train, y_train)
rf_regressor = rf_search.best_estimator_

y_pred_rf = rf_regressor.predict(X_valid)


## 4. 검증 데이터를 이용한 평가

In [ ]:

from sklearn.metrics import mean_absolute_error

# 기존 모델 평가
mae_dt = mean_absolute_error(y_valid, y_pred_dt)
print("Decision Tree Regressor")
print(f"Mean Absolute Error: {mae_dt:.4f}")

mae_svm = mean_absolute_error(y_valid, y_pred_svm)
print("Support Vector Machine Regressor")
print(f"Mean Absolute Error: {mae_svm:.4f}")

# 추가 모델 평가
mae_rf = mean_absolute_error(y_valid, y_pred_rf)
print("Random Forest Regressor")
print(f"Mean Absolute Error: {mae_rf:.4f}")


Public dataset score
Decision Tree Regressor
Mean Absolute Error: 1.8652
Support Vector Machine Regressor
Mean Absolute Error: 1.3040


## 5. 제출 파일 생성

In [ ]:

# 각 모델의 MAE 기반 가중치 계산
total_mae = 1 / mae_dt + 1 / mae_svm + 1 / mae_rf
w_dt = (1 / mae_dt) / total_mae
w_svm = (1 / mae_svm) / total_mae
w_rf = (1 / mae_rf) / total_mae

print(f"Dynamic Weights -> DT: {w_dt:.2f}, SVM: {w_svm:.2f}, RF: {w_rf:.2f}")

# 가중치 기반 앙상블
final_predictions = (w_dt * dt_regressor.predict(X_test_scaled) +
                     w_svm * svm_regressor.predict(X_test_scaled) +
                     w_rf * rf_regressor.predict(X_test_scaled))

# 제출 파일 생성
results = pd.DataFrame({
    'id': range(len(final_predictions)),
    'Age': final_predictions
})
results.to_csv('final_submission.csv', index=False)

from google.colab import files
files.download('final_submission.csv')